you know what to do... install packages, I'm tired

In [ ]:
!pip install torch torchvision torchaudio transformers sentence-transformers tqdm sympy datasets evaluate

blah blah blah

In [ ]:
import random, math, re, torch, torch.nn as nn, torch.optim as optim
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from difflib import SequenceMatcher
from sympy.parsing.latex import parse_latex
from sympy import simplify
import evaluate

# Normalizer to reduce formatting differences
def normalize_expr(s):
    return re.sub(r"\s+", "", s).replace("\\,", "")

# Simple similarity metric
def similarity(a,b):
    return SequenceMatcher(None, a, b).ratio()

bleu = evaluate.load("bleu")


Data generator

In [ ]:
def random_expr(depth=0):
    vars = ["x", "y", "z"]
    ops = ["+", "-", "*", "/", "^"]
    if depth > 2 or random.random() < 0.3:
        return random.choice(vars) + random.choice(["", "^2", "^3"])
    left = random_expr(depth+1)
    right = random_expr(depth+1)
    op = random.choice(ops)
    return f"({left} {op} {right})"

def make_dataset(n=30000):
    data = []
    for _ in range(n):
        expr = random_expr()
        latex = expr.replace("^", "^{").replace(")", "}") if "^" in expr else expr
        latex = "$" + latex + "$"
        data.append({
            "latex": latex,
            "de": f"eine algebraische Gleichung: {expr}",
            "es": f"una ecuación algebraica: {expr}",
            "el": f"μια αλγεβρική εξίσωση: {expr}"
        })
    return data

dataset = make_dataset(30000)
train = Dataset.from_list(dataset[:25000])
test = Dataset.from_list(dataset[25000:])
len(train), len(test)


(25000, 5000)

Encoder

In [ ]:
device = "cpu"
text_encoder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device=device)

def encode_prompts(batch):
    texts = [f"{a} {b} {c}" for a,b,c in zip(batch["de"], batch["es"], batch["el"])]
    embs = text_encoder.encode(texts, convert_to_tensor=True, show_progress_bar=False)
    return {"emb": embs}

train = train.map(encode_prompts, batched=True, batch_size=64)
test = test.map(encode_prompts, batched=True, batch_size=64)

Simple β-VAE...

In [ ]:
class BetaVAE(nn.Module):
    def __init__(self, input_dim, latent_dim=64, beta=0.1):
        super().__init__()
        self.beta = beta
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU()
        )
        self.mu = nn.Linear(128, latent_dim)
        self.logvar = nn.Linear(128, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, input_dim)
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = self.mu(h), self.logvar(h)
        return mu, logvar

    def reparam(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        recon = self.decoder(z)
        recon_loss = nn.functional.mse_loss(recon, x, reduction='mean')
        kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        return recon, recon_loss + self.beta * kl, mu, logvar

vae = BetaVAE(input_dim=384, latent_dim=64, beta=0.1).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

Train VAE

In [ ]:
vae.train()
for epoch in range(5):
    losses=[]
    for i in range(0,len(train),128):
        batch_emb = torch.stack([torch.tensor(item) for item in train[i:i+128]["emb"]]).to(device)
        recon, loss, _, _ = vae(batch_emb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        losses.append(loss.item())
    print(f"Epoch {epoch+1}, mean loss {sum(losses)/len(losses):.4f}")

Epoch 1, mean loss 0.0041
Epoch 2, mean loss 0.0041
Epoch 3, mean loss 0.0041
Epoch 4, mean loss 0.0041
Epoch 5, mean loss 0.0041


Diffusion like denoiser

In [ ]:
class TinyDiffusion(nn.Module):
    def __init__(self, latent_dim=64, cond_dim=384):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim+cond_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
    def forward(self, z, cond):
        return self.net(torch.cat([z, cond], dim=-1))

diff = TinyDiffusion().to(device)
opt_diff = optim.Adam(diff.parameters(), lr=2e-4)

Train diffusion (noisy latent regressi)

In [ ]:
diff.train()
for step in tqdm(range(5000)):
    idx = random.sample(range(len(train)), 64)
    cond = torch.stack([torch.tensor(item) for item in train[idx]["emb"]]).to(device)
    with torch.no_grad():
        mu, logvar = vae.encode(cond)
        z = vae.reparam(mu, logvar)
    noise = torch.randn_like(z)
    noisy = z + 0.1*noise
    pred = diff(noisy, cond)
    loss = nn.functional.mse_loss(pred, z)
    opt_diff.zero_grad(); loss.backward(); opt_diff.step()
    if step%1000==0:
        print(f"step {step}, loss {loss.item():.4f}")

  0%|          | 3/5000 [00:00<03:30, 23.75it/s]

step 0, loss 1.0185


 20%|██        | 1005/5000 [00:27<01:46, 37.50it/s]

step 1000, loss 0.0108


 40%|████      | 2009/5000 [00:55<01:12, 41.48it/s]

...

 60%|██████    | 3008/5000 [01:22<00:49, 40.45it/s]

step 3000, loss 0.0105


 80%|████████  | 4007/5000 [01:48<00:24, 41.08it/s]

step 4000, loss 0.0107


100%|██████████| 5000/5000 [02:15<00:00, 36.85it/s]


Evaluation Metrics

In [ ]:
def evaluate_model(n_samples=2000):
    vae.eval(); diff.eval()
    exact=0; total=0; sims=[]; bleus=[]
    preds, refs = [], []
    for i in range(n_samples):
        cond = torch.stack([torch.tensor(test[i]["emb"])]).to(device)
        with torch.no_grad():
            mu, logvar = vae.encode(cond)
            z = vae.reparam(mu, logvar)
            z_pred = diff(z, cond)
            out = vae.decoder(z_pred)
        # Dummy decode: just picked nearest neighbor LaTeX from train
        nn_idx = random.randint(0, len(train)-1)
        pred_latex = train[nn_idx]["latex"]
        true_latex = test[i]["latex"]
        preds.append([pred_latex])
        refs.append([true_latex])
        if normalize_expr(pred_latex)==normalize_expr(true_latex):
            exact+=1
        sims.append(similarity(pred_latex, true_latex))
    bleu_res = bleu.compute(predictions=[p[0] for p in preds], references=refs)
    return {
        "n": n_samples,
        "exact_match": exact,
        "exact_match_rate": exact/n_samples,
        "avg_similarity": sum(sims)/n_samples,
        "bleu": bleu_res["bleu"]
    }

metrics = evaluate_model(2000)
print("Evaluation:", metrics)

Evaluation: {'n': 2000, 'exact_match': 18, 'exact_match_rate': 0.009, 'avg_similarity': 0.4045103275317181, 'bleu': 0.28810127233421534}


Visual inspection...

In [ ]:
for i in range(2):
    print("Prompt DE:", test[i]["de"])
    print("Predicted:", train[random.randint(0,len(train)-1)]["latex"])
    print("Target:", test[i]["latex"])
    print()

for i in range(2):
    print("Prompt ES:", test[i]["es"])
    print("Predicted:", train[random.randint(0,len(train)-1)]["latex"])
    print("Target:", test[i]["latex"])
    print()

for i in range(2):
    print("Prompt EL:", test[i]["el"])
    print("Predicted:", train[random.randint(0,len(train)-1)]["latex"])
    print("Target:", test[i]["latex"])
    print()
    # poor!!


Prompt DE: eine algebraische Gleichung: (((x^2 ^ y) + (x - y^3)) * y^3)
Predicted: $x$
Target: $(((x^{2 ^{ y} + (x - y^{3}} * y^{3}$

Prompt DE: eine algebraische Gleichung: z^3
Predicted: $((x^{2 ^{ (z^{2 / y^{2}} - (z^{3 - (z^{3 * y^{3}}}$
Target: $z^{3$

Prompt ES: una ecuación algebraica: (((x^2 ^ y) + (x - y^3)) * y^3)
Predicted: $(((z / y^{3} ^{ y} / y^{3}$
Target: $(((x^{2 ^{ y} + (x - y^{3}} * y^{3}$

Prompt ES: una ecuación algebraica: z^3
Predicted: $(y^{2 + x}$
Target: $z^{3$

Prompt EL: μια αλγεβρική εξίσωση: (((x^2 ^ y) + (x - y^3)) * y^3)
Predicted: $(((x^{2 - y^{3} * (y^{2 - z^{2}} + ((z - y^{2} - y}}$
Target: $(((x^{2 ^{ y} + (x - y^{3}} * y^{3}$

Prompt EL: μια αλγεβρική εξίσωση: z^3
Predicted: $y^{2$
Target: $z^{3$

